In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pickle import dump
import pickle as pkl
from statsmodels.distributions.empirical_distribution import ECDF
import copy
import json
import gzip

import os
import sys

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

FS_MOL_CHECKOUT_PATH = os.path.join('/Users/andrew/Code/ersilia/ersilia-fsmol/') # TODO set path to FS-Mol repo
FS_MOL_DATASET_PATH = os.path.join('/Users/andrew/Code/ersilia/bioassays/dataset/min_size_16') # TODO set path to FS-Mol repo

os.chdir(FS_MOL_CHECKOUT_PATH)
sys.path.insert(0, FS_MOL_CHECKOUT_PATH)

from fs_mol.data import FSMolDataset, DataFold

# Train set

In [70]:
dataset = FSMolDataset.from_directory(FS_MOL_DATASET_PATH)

task_iterable_train = dataset.get_task_reading_iterable(DataFold.TRAIN)

In [71]:
# Task names
tasks_train = list()

for task in iter(task_iterable_train):
    tasks_train.append(task.name)
    
# Task names to id dict
tasks_train_id_dict = {}
for i in range(len(tasks_train)):
    tasks_train_id_dict[tasks_train[i]] = i

In [72]:
# Make mol_id by comparing canonical smiles
# Make triplett: Mol_id, Task_id, Labels

mol_ids = list()
task_ids = list()
labels = list()

train_smiles_molId_dict = dict()
id_counter = 0

fingerprints = dict()
descriptors = dict()

for task in iter(task_iterable_train):
    for mol_idx in range(len(task.samples)):
        
        if task.samples[mol_idx].smiles not in list(train_smiles_molId_dict.keys()):
            train_smiles_molId_dict[task.samples[mol_idx].smiles] = id_counter
            id_counter += 1
            
        mol_ids.append(train_smiles_molId_dict[task.samples[mol_idx].smiles])
        task_ids.append(tasks_train_id_dict[task.name])
        labels.append(task.samples[mol_idx].bool_label)
        
        if task.samples[mol_idx].smiles not in list(fingerprints.keys()):
            fingerprints[task.samples[mol_idx].smiles] = task.samples[mol_idx].fingerprint
            descriptors[task.samples[mol_idx].smiles] = task.samples[mol_idx].descriptors
        
            
            

In [73]:
# Make numpy arrays for fingerprints and descriptors

fingerprints_temp = dict()
for key,value in zip(fingerprints.keys(),fingerprints.values()):
    fingerprints_temp[train_smiles_molId_dict[key]] = value

descriptors_temp = dict()
for key,value in zip(descriptors.keys(),descriptors.values()):
    descriptors_temp[train_smiles_molId_dict[key]] = value

fingerprints = np.array(list(fingerprints_temp.values()))
descriptors = np.array(list(descriptors_temp.values()))

In [74]:
# Compute quantils for descriptors
descriptors_raw_forECDF = copy.deepcopy(descriptors)
#TODO change path
# make a directory if it doesn't exist
if not os.path.exists('/Users/andrew/Code/ersilia/ersilia-mhnfs/data/preprocessed_datasets/training'):
    os.makedirs('/Users/andrew/Code/ersilia/ersilia-mhnfs/data/preprocessed_datasets/training')
np.save('/Users/andrew/Code/ersilia/ersilia-mhnfs/data/preprocessed_datasets/training/descriptors_raw_forECDF.npy',
        descriptors_raw_forECDF)

descriptors_quantils = np.zeros_like(descriptors_raw_forECDF)

for column in range(descriptors_raw_forECDF.shape[1]):
    raw_values = descriptors_raw_forECDF[:,column].reshape(-1)
    ecdf = ECDF(raw_values)
    quantils = ecdf(raw_values)
    
    descriptors_quantils[:, column] = quantils
    
    

In [75]:
# Make numpy array: mol_inputs
mol_inputs = np.hstack([fingerprints, descriptors_quantils])

print(mol_inputs.shape)
mol_inputs[0:5,:]

# TODO: for now, we are cutting off the last 10 columns
mol_inputs = mol_inputs[:, :-10]
print(mol_inputs.shape)

(16214, 2258)
(16214, 2248)


In [76]:
# Normalize mol_inputs and save scaler
mol_inputs[mol_inputs.astype('str') == 'nan'] = 0
mol_inputs[mol_inputs.astype('str') == 'inf'] = 0

scaler = StandardScaler()
scaler.fit(mol_inputs)
#TODO change path
dump(scaler, open('/Users/andrew/Code/ersilia/ersilia-mhnfs/data/preprocessed_datasets/scaler_trainFitted.pkl', 'wb'))

In [77]:
mol_inputs = scaler.transform(mol_inputs)

In [78]:
# Active dict
triplett_ds = pd.DataFrame({'mol':mol_ids,
                            'task':task_ids,
                            'labels':labels})

task_actives = dict()
task_inactives = dict()

for task in np.unique(task_ids):
    subset_task = triplett_ds[triplett_ds['task'] == task]
    subset_actives = subset_task[subset_task['labels'] == True]
    subset_inactives = subset_task[subset_task['labels'] == False]
    
    set_actives = list(subset_actives['mol'])
    set_inactives = list(subset_inactives['mol'])
    if len(set_actives) == 0:
        print("empty list of actives!")
        continue
        # raise ValueError('Active set: Empty list!')
    if len(set_inactives) == 0:
        print("empty list of inactives!")
        continue
        # raise ValueError('Inactive set: Empty list!')
    
    # TODO: figure out why some tasks have no actives / inactives
    task_actives[task] = set_actives
    task_inactives[task] = set_inactives
    

In [79]:
# Save files
# molecular features
#TODO change path
path_preproc = '/Users/andrew/Code/ersilia/ersilia-mhnfs/data/preprocessed_datasets'
np.save(f'{path_preproc}/training/mol_inputs.npy', mol_inputs)

# Tripletts
mol_ids = np.array(mol_ids).reshape(-1,1)
task_ids = np.array(task_ids).reshape(-1,1)
labels = np.array(labels).reshape(-1,1)
#TODO change path
np.save(f'{path_preproc}/training/mol_ids.npy', mol_ids)
#TODO change path
np.save(f'{path_preproc}/training/task_ids.npy', task_ids)
#TODO change path
np.save(f'{path_preproc}/training/labels.npy', labels)

# Dicts
#TODO change path
dump(tasks_train_id_dict, open(f'{path_preproc}/training/'
                               'dict_task_names_id.pkl', 'wb'))
#TODO change path
dump(train_smiles_molId_dict, open(f'{path_preproc}/training/'
                               'dict_mol_smiles_id.pkl', 'wb'))
#TODO change path
dump(task_actives, open(f'{path_preproc}/training/'
                               'dict_task_id_activeMolecules.pkl', 'wb'))
#TODO change path
dump(task_inactives, open(f'{path_preproc}/training/'
                               'dict_task_id_inactiveMolecules.pkl', 'wb'))

# Val set

In [80]:
dataset = FSMolDataset.from_directory(FS_MOL_DATASET_PATH)
task_iterable = dataset.get_task_reading_iterable(DataFold.VALIDATION)

In [81]:
# Task names
tasks = list()

for task in iter(task_iterable):
    tasks.append(task.name)
    
# Task names to id dict
tasks_id_dict = {}
for i in range(len(tasks)):
    tasks_id_dict[tasks[i]] = i

In [82]:
# Make mol_id by comparing canonical smiles
# Make triplett: Mol_id, Task_id, Labels

mol_ids = list()
task_ids = list()
labels = list()

smiles_molId_dict = dict()
id_counter = 0

fingerprints = dict()
descriptors = dict()

for task in iter(task_iterable):
    for mol_idx in range(len(task.samples)):
        
        if task.samples[mol_idx].smiles not in list(smiles_molId_dict.keys()):
            smiles_molId_dict[task.samples[mol_idx].smiles] = id_counter
            id_counter += 1
            
        mol_ids.append(smiles_molId_dict[task.samples[mol_idx].smiles])
        task_ids.append(tasks_id_dict[task.name])
        labels.append(task.samples[mol_idx].bool_label)
        
        if task.samples[mol_idx].smiles not in list(fingerprints.keys()):
            fingerprints[task.samples[mol_idx].smiles] = task.samples[mol_idx].fingerprint
            descriptors[task.samples[mol_idx].smiles] = task.samples[mol_idx].descriptors

In [83]:
# Make numpy array: mol_inputs

fingerprints_temp = dict()
for key,value in zip(fingerprints.keys(),fingerprints.values()):
    fingerprints_temp[smiles_molId_dict[key]] = value

descriptors_temp = dict()
for key,value in zip(descriptors.keys(),descriptors.values()):
    descriptors_temp[smiles_molId_dict[key]] = value

fingerprints = np.array(list(fingerprints_temp.values()))
descriptors = np.array(list(descriptors_temp.values()))

# Compute quantils for descriptors
descriptors_raw_forECDF = copy.deepcopy(descriptors)
#TODO change path
# make a directory if it doesn't exist
if not os.path.exists(f'{path_preproc}/validation'):
    os.makedirs(f'{path_preproc}/validation')
np.save(f'{path_preproc}/validation/descriptors_raw_forECDF.npy',
        descriptors_raw_forECDF)

descriptors_quantils = np.zeros_like(descriptors_raw_forECDF)

for column in range(descriptors_raw_forECDF.shape[1]):
    raw_values = descriptors_raw_forECDF[:,column].reshape(-1)
    ecdf = ECDF(raw_values)
    quantils = ecdf(raw_values)
    
    descriptors_quantils[:, column] = quantils

mol_inputs = np.hstack([fingerprints, descriptors_quantils])

print(mol_inputs.shape)
mol_inputs[0:5,:]

# TODO: for now, we are cutting off the last 10 columns
mol_inputs = mol_inputs[:, :-10]
print(mol_inputs.shape)

(837, 2258)
(837, 2248)


In [84]:
# Normalize mol_inputs
mol_inputs[mol_inputs.astype('str') == 'nan'] = 0
mol_inputs[mol_inputs.astype('str') == 'inf'] = 0

#TODO change path
scaler = pkl.load(open(f'{path_preproc}/scaler_trainFitted.pkl',
                       'rb'))

mol_inputs = scaler.transform(mol_inputs)

In [85]:
# Active dict
triplett_ds = pd.DataFrame({'mol':mol_ids,
                            'task':task_ids,
                            'labels':labels})

task_actives = dict()
task_inactives = dict()

for task in np.unique(task_ids):
    subset_task = triplett_ds[triplett_ds['task'] == task]
    subset_actives = subset_task[subset_task['labels'] == True]
    subset_inactives = subset_task[subset_task['labels'] == False]
    
    set_actives = list(subset_actives['mol'])
    set_inactives = list(subset_inactives['mol'])
    if len(set_actives) == 0:
        raise ValueError('Active set: Empty list!')
    if len(set_inactives) == 0:
        raise ValueError('Inactive set: Empty list!')
    
    task_actives[task] = set_actives
    task_inactives[task] = set_inactives

In [86]:
# Save files
# molecular features
#TODO change path
np.save(f'{path_preproc}/validation/mol_inputs.npy', mol_inputs)

# Tripletts
mol_ids = np.array(mol_ids).reshape(-1,1)
task_ids = np.array(task_ids).reshape(-1,1)
labels = np.array(labels).reshape(-1,1)
#TODO change path
np.save(f'{path_preproc}/validation/mol_ids.npy', mol_ids)
#TODO change path
np.save(f'{path_preproc}/validation/task_ids.npy', task_ids)
#TODO change path
np.save(f'{path_preproc}/validation/labels.npy', labels)

# Dicts
#TODO change path
dump(tasks_id_dict, open(f'{path_preproc}/validation/'
                               'dict_task_names_id.pkl', 'wb'))
#TODO change path
dump(smiles_molId_dict, open(f'{path_preproc}/validation/'
                               'dict_mol_smiles_id.pkl', 'wb'))
#TODO change path
dump(task_actives, open(f'{path_preproc}/validation/'
                               'dict_task_id_activeMolecules.pkl', 'wb'))
#TODO change path
dump(task_inactives, open(f'{path_preproc}/validation/'
                               'dict_task_id_inactiveMolecules.pkl', 'wb'))

# Test set

In [87]:
dataset = FSMolDataset.from_directory(FS_MOL_DATASET_PATH)
task_iterable = dataset.get_task_reading_iterable(DataFold.TEST)

In [88]:
# Task names
tasks = list()

for task in iter(task_iterable):
    tasks.append(task.name)
    
# Task names to id dict
tasks_id_dict = {}
for i in range(len(tasks)):
    tasks_id_dict[tasks[i]] = i

In [89]:
# Make mol_id by comparing canonical smiles
# Make triplett: Mol_id, Task_id, Labels

mol_ids = list()
task_ids = list()
labels = list()

smiles_molId_dict = dict()
id_counter = 0

fingerprints = dict()
descriptors = dict()

for task in iter(task_iterable):
    for mol_idx in range(len(task.samples)):
        
        if task.samples[mol_idx].smiles not in list(smiles_molId_dict.keys()):
            smiles_molId_dict[task.samples[mol_idx].smiles] = id_counter
            id_counter += 1
            
        mol_ids.append(smiles_molId_dict[task.samples[mol_idx].smiles])
        task_ids.append(tasks_id_dict[task.name])
        labels.append(task.samples[mol_idx].bool_label)
        
        if task.samples[mol_idx].smiles not in list(fingerprints.keys()):
            fingerprints[task.samples[mol_idx].smiles] = task.samples[mol_idx].fingerprint
            descriptors[task.samples[mol_idx].smiles] = task.samples[mol_idx].descriptors

In [90]:
# Make numpy array: mol_inputs

fingerprints_temp = dict()
for key,value in zip(fingerprints.keys(),fingerprints.values()):
    fingerprints_temp[smiles_molId_dict[key]] = value

descriptors_temp = dict()
for key,value in zip(descriptors.keys(),descriptors.values()):
    descriptors_temp[smiles_molId_dict[key]] = value

fingerprints = np.array(list(fingerprints_temp.values()))
descriptors = np.array(list(descriptors_temp.values()))

# Compute quantils for descriptors
descriptors_quantils = np.zeros_like(descriptors)

for column in range(descriptors_raw_forECDF.shape[1]):
    raw_values_ecdf = descriptors_raw_forECDF[:,column].reshape(-1)
    raw_values = descriptors[:,column].reshape(-1)
    
    ecdf = ECDF(raw_values_ecdf)
    quantils = ecdf(raw_values)
    
    descriptors_quantils[:, column] = quantils

mol_inputs = np.hstack([fingerprints, descriptors_quantils])

print(mol_inputs.shape)
mol_inputs[0:5,:]

# TODO: for now, we are cutting off the last 10 columns
mol_inputs = mol_inputs[:, :-10]
print(mol_inputs.shape)

(4494, 2258)
(4494, 2248)


In [91]:
# Normalize mol_inputs
mol_inputs[mol_inputs.astype('str') == 'nan'] = 0
mol_inputs[mol_inputs.astype('str') == 'inf'] = 0

#TODO change path
scaler = pkl.load(open(f'{path_preproc}/scaler_trainFitted.pkl', 'rb'))

mol_inputs = scaler.transform(mol_inputs)

In [92]:
# Active dict
triplett_ds = pd.DataFrame({'mol':mol_ids,
                            'task':task_ids,
                            'labels':labels})

task_actives = dict()
task_inactives = dict()

for task in np.unique(task_ids):
    subset_task = triplett_ds[triplett_ds['task'] == task]
    subset_actives = subset_task[subset_task['labels'] == True]
    subset_inactives = subset_task[subset_task['labels'] == False]
    
    set_actives = list(subset_actives['mol'])
    set_inactives = list(subset_inactives['mol'])
    if len(set_actives) == 0:
        raise ValueError('Active set: Empty list!')
    if len(set_inactives) == 0:
        raise ValueError('Inactive set: Empty list!')
    
    task_actives[task] = set_actives
    task_inactives[task] = set_inactives

In [93]:
# Save files
# molecular features
#TODO change path
if not os.path.exists(f'{path_preproc}/test/'):
    os.makedirs(f'{path_preproc}/test/')
np.save(f'{path_preproc}/test/mol_inputs.npy', mol_inputs)

# Tripletts
mol_ids = np.array(mol_ids).reshape(-1,1)
task_ids = np.array(task_ids).reshape(-1,1)
labels = np.array(labels).reshape(-1,1)
#TODO change path
np.save(f'{path_preproc}/test/mol_ids.npy', mol_ids)
#TODO change path
np.save(f'{path_preproc}/test/task_ids.npy', task_ids)
#TODO change path
np.save(f'{path_preproc}/test/labels.npy', labels)

# Dicts
#TODO change path
dump(tasks_id_dict, open(f'{path_preproc}/test/'
                               'dict_task_names_id.pkl', 'wb'))
#TODO change path
dump(smiles_molId_dict, open(f'{path_preproc}/test/'
                               'dict_mol_smiles_id.pkl', 'wb'))
#TODO change path
dump(task_actives, open(f'{path_preproc}/test/'
                               'dict_task_id_activeMolecules.pkl', 'wb'))
#TODO change path
dump(task_inactives, open(f'{path_preproc}/test/'
                               'dict_task_id_inactiveMolecules.pkl', 'wb'))